In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install segmentation-models-pytorch
!pip install rasterio
!pip install richdem

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from pathlib import Path
import shutil
import pandas as pd  
import richdem as rd

#1. Uncomment the section with your name above it

#2. Run the code and document the results

In [5]:
# Root Directory
Image_Segmentation_Path = '/content/drive/My Drive/Image Segmentation/'

# Inputs/Sources

dataframes_path = Image_Segmentation_Path + "DataFrames/"

# This is the entire dataset. We'll do it later.
# dirs_pairs = dataframes_path + "complete_dirs_pairs.csv"
# train_dirs_path = dataframes_path + 'complete_train_dirs.csv'
# validate_dirs_path = dataframes_path + 'complete_validate_dirs.csv'
# test_dirs_path = dataframes_path + 'complete_test_dris.csv'

# Aditya
#dirs_pairs = dataframes_path + "origin_dirs_pairs.csv"
#train_dirs_path = dataframes_path + 'origin_train_dirs.csv'
#validate_dirs_path = dataframes_path + 'origin_validate_dirs.csv'
# test_dirs_path = dataframes_path + 'origin_test_dris.csv'


# Douglas
#dirs_pairs = dataframes_path + "slope_dirs_pairs.csv"
#train_dirs_path = dataframes_path + 'slope_train_dirs.csv'
#validate_dirs_path = dataframes_path + 'slope_validate_dirs.csv'
#test_dirs_path = dataframes_path + 'slope_test_dris.csv'


# Seif
#dirs_pairs = dataframes_path + "aspect_dirs_pairs.csv"
#train_dirs_path = dataframes_path + 'aspect_train_dirs.csv'
#validate_dirs_path = dataframes_path + 'aspect_validate_dirs.csv'
#test_dirs_path = dataframes_path + 'aspect_test_dris.csv'

# Salim
#dirs_pairs = dataframes_path + "hillshade_dirs_pairs.csv"
#train_dirs_path = dataframes_path + 'hillshade_train_dirs.csv'
#validate_dirs_path = dataframes_path + 'hillshade_validate_dirs.csv'
#test_dirs_path = dataframes_path + 'hillshade_test_dris.csv'


In [6]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
from torch import FloatTensor

In [7]:
class Dataset(BaseDataset):
  """
  Args:
      images_dir (str): path to images folder
      masks_dir (str): path to segmentation masks folder
      class_values (list): values of classes to extract from segmentation mask
  """
    
  CLASSES = ['non-mound', 'mound', 'invalid data']
    
  def __init__(
          self, 
          dirs,
          classes=None,
          augmentation=None):
    self.df = pd.read_csv(dirs)
    self.ids = self.df['Index'].values.tolist()
    
    # convert str names to class values on masks
    self.class_values = [0, 1, -1]
      
      
    self.augmentation = augmentation
    
  def __getitem__(self, i):
    # read data
    with rasterio.open(self.df.at[i, 'Input']) as dem:
      dem_array = dem.read(1)
      
    with rasterio.open(self.df.at[i, 'Target']) as mask:
      mask_array = mask.read(1)

    # apply augmentations
    if self.augmentation:
      sample = self.augmentation(image=dem_array, mask=mask_array)
      dem_array, mask_array = sample['image'], sample['mask']
    
    dem_array = FloatTensor(np.expand_dims(dem_array,axis=(0)))
    mask_array = FloatTensor(np.expand_dims(mask_array,axis=(0)))
    
    return dem_array, mask_array
      
  def __len__(self):
    return len(self.ids)

In [8]:
dataset = Dataset(dirs_pairs, classes=['mound'])
dem_array, mask_array = dataset[1] # get some sample

In [9]:
mask_array.shape

torch.Size([1, 201, 177])

In [10]:
import albumentations as albu

In [11]:
def get_training_augmentation():
  train_transform = [albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0)]

  return albu.Compose(train_transform)


In [12]:
def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(384, 480)
    ]
    return albu.Compose(test_transform)

In [13]:
import torch
import numpy as np
import segmentation_models_pytorch as smp

In [14]:
# ENCODER = 'se_resnext50_32x4d'
# ENCODER_WEIGHTS = 'imagenet'
# ACTIVATION = 'softmax2d' # could be None for logits or 'softmax2d' for multiclass segmentation
# CLASSES = ['non-mound', 'mound', 'invalid data']
# # create segmentation model with pretrained encoder
# model = smp.FPN(
#     encoder_name=ENCODER, 
#     encoder_weights=ENCODER_WEIGHTS, 
#     classes=len(CLASSES), 
#     activation=ACTIVATION,
# )

In [15]:
CLASSES = ['non-mound', 'mound', 'invalid data']
model = smp.Unet(
    encoder_name="resnet34",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    classes=len(CLASSES),
    in_channels=1,
    activation='softmax2d'                  # model output channels (number of classes in your dataset)
)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [16]:
train_dataset = Dataset(train_dirs_path,
                        augmentation=get_training_augmentation(),                         
                        classes=CLASSES)


valid_dataset = Dataset(validate_dirs_path,
                        augmentation=get_validation_augmentation(),                         
                        classes=CLASSES)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=2)

In [17]:
dem, mask = train_dataset[6]
dem.shape

torch.Size([1, 320, 320])

In [18]:
torch.max(dem)

tensor(21.0122)

In [19]:
# Dice/F1 score - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
# IoU/Jaccard score - https://en.wikipedia.org/wiki/Jaccard_index

loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.01),
])

In [20]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    verbose=True,
)

In [ ]:
# train model for 40 epochs

max_score = 0

for i in range(0, 40):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, './best_model.pth')
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-3
        print('Decrease decoder learning rate to 1e-3!')


Epoch: 0
valid: 100%|██████████| 546/546 [07:57<00:00,  1.14it/s, dice_loss - 6.347e+04, iou_score - 7.758]
Model saved!

Epoch: 1
valid: 100%|██████████| 546/546 [08:00<00:00,  1.14it/s, dice_loss - 6.347e+04, iou_score - -0.4633]

Epoch: 2
valid: 100%|██████████| 546/546 [08:08<00:00,  1.12it/s, dice_loss - 6.347e+04, iou_score - 0.6235]

Epoch: 3
valid: 100%|██████████| 546/546 [08:12<00:00,  1.11it/s, dice_loss - 6.347e+04, iou_score - 69.42]
Model saved!

Epoch: 4
valid: 100%|██████████| 546/546 [08:19<00:00,  1.09it/s, dice_loss - 6.347e+04, iou_score - 0.1016]

Epoch: 5
valid: 100%|██████████| 546/546 [08:19<00:00,  1.09it/s, dice_loss - 6.347e+04, iou_score - 1.871]

Epoch: 6
valid: 100%|██████████| 546/546 [08:16<00:00,  1.10it/s, dice_loss - 6.347e+04, iou_score - 12.2]

Epoch: 7
train:  72%|███████▏  | 148/205 [31:06<12:08, 12.77s/it, dice_loss - 1.137, iou_score - -0.0454]

In [ ]:
# load best saved checkpoint
best_model = torch.load('./best_model.pth')

In [ ]:
# create test dataset
test_dataset = Dataset(test_dirs_path,
                       augmentation=get_validation_augmentation(), 
                       preprocessing=get_preprocessing(preprocessing_fn),
                       classes=CLASSES)

test_dataloader = DataLoader(test_dataset)

In [ ]:
# evaluate model on test set
test_epoch = smp.utils.train.ValidEpoch(
    model=best_model,
    loss=loss,
    metrics=metrics)

logs = test_epoch.run(test_dataloader)

In [ ]:
# for i in range(5):
#     n = np.random.choice(len(test_dataset))
    
#     image_vis = test_dataset[n][0].astype('uint8')
#     image, gt_mask = test_dataset[n]
    
    
#     x_tensor = torch.from_numpy(image).unsqueeze(0)
#     pr_mask = best_model.predict(x_tensor)
#     pr_mask = (pr_mask.cpu().numpy().round())
        
#     visualize(
#         image=image_vis, 
#         ground_truth_mask=gt_mask, 
#         predicted_mask=pr_mask
#     )